In [93]:
import torch
import itertools
from scipy.special import comb

class LDPIndepTester:
    def __init__(self, cuda_device):
        self.cuda_device = cuda_device

   
    def bin_separately(self, data_X, data_Y, kappa):
        return (
            self.bin(data_X, kappa),
            self.bin(data_Y, kappa)
            )

        
    def range_check(self, data):
        if (torch.sum(data.gt(1))).gt(0):
            print("check data range")
            return False
        elif (torch.sum(data.le(0))).gt(0):
            print("check data range")
            return False
        else:
            return True

    def run_test_conti_data(self, B, data_Y, data_Z, kappa, alpha, gamma, discrete_noise = False):
        #0. data range check
        
        if not self.range_check(data_Y):
            return
        if not self.range_check(data_Z):
            return
        
        #1. bin
        n = data_Y.size(dim = 0)
        data_Y_binned, data_Z_binned = self.bin_separately(data_Y, data_Z, kappa)

        #2. privatize
        data_Y_priv, data_Z_priv, noise_var_Y, noise_var_Z = self.privatize_indep(
            data_Y = data_Y,
            data_Z = data_Z,
            alpha = alpha,
            discrete_noise = discrete_noise
        )
        
        #4 compute original u-stat
        ustatOriginal = self.u_stat_indep_matrix(data_Y_priv, data_Z_priv)

        print(f"original u-statistic:{ustatOriginal}")
        
        #permutation procedure
        permStats = torch.empty(B).to(self.cuda_device)
        
        for i in range(B):
            perm_stat_now = self.u_stat_indep_matrix(
                data_Y_priv,
                data_Z_priv[
                    torch.randperm(data_Z_priv.size(dim=0))],
                ).to(self.cuda_device)

            permStats[i] = perm_stat_now
            #print(perm_stat_now)
         
        
        p_value_proxy = (1 +
                         torch.sum(
                             torch.gt(input = permStats, other = ustatOriginal)
                         )
                        ) / (B + 1)
        
        
        print(f"p value proxy: {p_value_proxy}")
        
        return(p_value_proxy < gamma, noise_var_Y, noise_var_Z)#test result: TRUE = 1 = reject the null, FALSE = 0 = retain the null.

        
    def privatize_indep(self, data_Y, data_Z, alpha = float("inf"), discrete_noise = False):
        ## assume the data is discrete by nature or has already been dicretized.
        n = data_Y.size(dim = 0) # Y and Z have the same sample size.
        kappa_d1 = data_Y.size(dim = 1) #kappa^d if conti data, d if discrete data
        kappa_d2 = data_Z.size(dim = 1) #kappa^d if conti data, d if discrete data

        print(f"noise dimension : {kappa_d1}, {kappa_d2}")
        
        scale_factor = torch.tensor( (kappa_d1 * kappa_d2)**(1/2) )
        sigma_kappa_alpha = 4 * (2 ** (1/2)) * scale_factor / alpha
        
        if alpha == float("inf"): #non-private case
            return( torch.mul(scale_factor, data) )
        else:
            data_Y_priv, noise_var_Y = self.privatize_indep_separate(
                    data = data_Y,
                    scale_factor = scale_factor,
                    sigma_kappa_alpha = sigma_kappa_alpha,
                    discrete_noise = discrete_noise
            )
            data_Z_priv, noise_var_Z = self.privatize_indep_separate(
                    data = data_Z,
                    scale_factor = scale_factor,
                    sigma_kappa_alpha = sigma_kappa_alpha,
                    discrete_noise = discrete_noise
            )
        return(data_Y_priv, data_Z_priv, noise_var_Y, noise_var_Z)
        
    
    def privatize_indep_separate(self, data, scale_factor, sigma_kappa_alpha, discrete_noise):
        n = data.size(dim = 0)
        if discrete_noise:
            noise, noise_var = self.noise_conti(data, sigma_kappa_alpha) #fix here later
        else:
            noise, noise_var = self.noise_conti(data, sigma_kappa_alpha)
        return(   
            torch.add(
                input = noise.reshape(n, -1),
                alpha = scale_factor,
                other = data
            ), 
            noise_var
        )
                       
    def noise_conti(self, data, sigma_kappa_alpha):
        #dim = kappa^d for conti data, d for discrete data
        laplace_samples = self.generate_unit_laplace(data)
        laplace_samples = sigma_kappa_alpha * laplace_samples
        print("noise type: conti")
        return( laplace_samples, torch.var(laplace_samples) )
    

    def generate_unit_laplace(self, data):
        n = data.size(dim = 0)
        d = data.size(dim = 1)
        unit_laplace_generator = torch.distributions.laplace.Laplace(
            torch.tensor(0.0).to(self.cuda_device),
            torch.tensor(2**(-1/2)).to(self.cuda_device)
        )
        return unit_laplace_generator.sample(sample_shape = torch.Size([n * d]))

  



##########################################################################################
################# For binning ############################################################
##########################################################################################
    def bin(self, data, kappa): 
        ''' Only for continuous data'''
        
        # create designated number of intervals
        d = self.get_dimension(data)
        print(d)
     
        # 1. for each dimension, turn the continuous data into interval
        # each row now indicates a hypercube in [0,1]^d
        # the more the data is closer to 1, the larger the interval index.
        dataInterval = self.transform_bin_index(data = data, nIntervals = kappa)
        
        # 2. for each datapoint(row),
        #    turn the hypercube data into a multivariate data of (1, 2, ..., kappa^d)
        #    each row now becomes an integer.
        dataMultivariate = self.TransformMultivariate(
            dataInterval = dataInterval,
            nBin = kappa,
        )
        # 3. turn the indices into one-hot vectors
        dataOnehot = self.TransformOnehot(dataMultivariate, kappa**d)
        return(dataOnehot)
    
    def transform_bin_index(self, data, nIntervals):
        ''' Only for continuous data.
        for each dimension, transform the data in [0,1] into the interval index
        first interval = [0, x], the others = (y z]
        
        input arguments
            data: torch tensor object on GPU
            nIntervals: integer
        output
            dataIndices: torch tensor, dimension same as the input
        '''
        # create designated number of intervals
        d = self.get_dimension(data)
        breaks = torch.linspace(start = 0, end = 1, steps = nIntervals + 1).to(self.cuda_device) #floatTensor
        dataIndices = torch.bucketize(data, breaks, right = False) # ( ] form.
        dataIndices = dataIndices.add(
            dataIndices.eq(0)
        ) #move 0 values from the bin number 0 to the bin number 1        
        return(dataIndices)
    
    def TransformMultivariate(self, dataInterval, nBin):
        """Only for continuous and multivariate data ."""
        d = self.get_dimension(dataInterval)
        
        if d == 1:
            return(dataInterval.sub(1))
        else:
            exponent = torch.linspace(start = (d-1), end = 0, steps = d, dtype = torch.long)
            vector = torch.tensor(nBin).pow(exponent)
            return( torch.matmul( dataInterval.sub(1).to(torch.float), vector.to(torch.float).to(self.cuda_device) ).to(torch.long) )
    
    def TransformOnehot(self, dataMultivariate, newdim):
        return(
            torch.nn.functional.one_hot(
                dataMultivariate,
                num_classes = newdim)
        )
    
    
    def get_dimension(self, data):
        if data.dim() == 1:
            return(1)
        elif data.dim() == 2:
            return( data.size(dim = 1) )
        else:
            return # we only use up to 2-dimensional tensor, i.e. matrix
##########################################################################################
##########################################################################################


    
    
    def kernel_indep(self, fourchunk):
        ip = torch.matmul(fourchunk, torch.transpose(fourchunk, 0, 1))
        return(ip[0,1] + ip[2,3] - ip[0,2] - ip[1,3])

     
    def u_stat_indep_matrix(self, data_X, data_Y):
        n = data_X.size(dim=0)
        Phi = torch.matmul(data_X, torch.transpose(data_X, 0, 1))
        Psi = torch.matmul(data_Y, torch.transpose(data_Y, 0, 1))
        Phi_tilde = Phi.fill_diagonal_(0.0)
        Psi_tilde = Psi.fill_diagonal_(0.0)
        n_four = n * (n-1) * (n-2) * (n-3)
        one = torch.ones(n, 1).to(device)
        oneT = torch.transpose(one, 0, 1)

        PhiPsi = torch.matmul(Phi, Psi)
        trPhiPsi = torch.trace(PhiPsi)
        onePhiPsiOne = torch.matmul(oneT, torch.matmul(PhiPsi, one))

        onePhione = torch.matmul(oneT, torch.matmul(Phi, one))
        onePsione = torch.matmul(oneT, torch.matmul(Psi, one))
        onePhioneonePsione = torch.matmul(onePhione, onePsione)

        Un = (
          4 * (onePhioneonePsione - 4 * onePhiPsiOne + 2 * trPhiPsi)
        - 8 * (n-3) *(onePhiPsiOne - trPhiPsi)
        + 4 * (n-3)*(n-2) * trPhiPsi
        )
        
        return(Un/n_four)
    
    def u_stat_indep_original(self, data_X, data_Y):
        n = data_X.size(dim = 0)
        print(f"number of calculation = {2*scipy.special.comb(n,4) }")
        n_four = n * (n-1) * (n-2) * (n-3)
        U_statistic = 0
        for i in range(n):
            set_j = set(range(n)) - {i}
            for j in set_j:
                set_k = set_j - {j}
                for k in set_k:
                    set_r = set_k - {k}
                    for r in set_r:
                        comb = [i,j,k,r]
                        U_statistic = U_statistic + (
                            self.kernel_indep(data_X[comb,]) * self.kernel_indep(data_Y[comb,])
                        )/n_four
        return(U_statistic)



In [97]:
import scipy
device = torch.device('cuda:1')

n= 1000
d = 2
loc = torch.zeros(d)
scale = torch.ones(d)*2
mvn1 = torch.distributions.MultivariateNormal(loc, scale_tril=torch.diag(scale))
mvn2 = torch.distributions.MultivariateNormal(loc, scale_tril=torch.diag(scale))

data_Y = mvn1.sample(sample_shape=torch.Size([n])).to(device)
data_Y = data_Y.abs() / data_Y.abs().max()

data_Z = mvn2.sample(sample_shape=torch.Size([n])).to(device)
data_Z = data_Z.abs() / data_Z.abs().max()

tester = LDPIndepTester(device)
tester.run_test_conti_data(B = 300, data_Y = data_Y, data_Z = data_Z, kappa = 3, alpha = 0.5, gamma = 0.05, discrete_noise = False)

2
2
noise dimension : 2, 2
noise type: conti
noise type: conti
original u-statistic:tensor([[-2381.4062]], device='cuda:1')
p value proxy: 0.8272424936294556


(tensor(False, device='cuda:1'),
 tensor(482.5581, device='cuda:1'),
 tensor(493.8383, device='cuda:1'))

dependent case

In [110]:
import scipy
device = torch.device('cuda:1')

n= 10
d = 2
loc = torch.zeros(d)
scale = torch.ones(d)*2
mvn1 = torch.distributions.MultivariateNormal(loc, scale_tril=torch.diag(scale))
mvn2 = torch.distributions.MultivariateNormal(loc, scale_tril=torch.diag(scale))

data_Y = mvn1.sample(sample_shape=torch.Size([n])).to(device)
data_Y = data_Y.abs() / data_Y.abs().max()

data_Z = data_Y

tester = LDPIndepTester(device)
tester.run_test_conti_data(B = 300, data_Y = data_Y, data_Z = data_Z, kappa = 3, alpha = 1, gamma = 0.05, discrete_noise = False)

2
2
noise dimension : 2, 2
noise type: conti
noise type: conti
original u-statistic:tensor([[-15337.7109]], device='cuda:1')
p value proxy: 0.6810631155967712


(tensor(False, device='cuda:1'),
 tensor(184.5706, device='cuda:1'),
 tensor(100.3413, device='cuda:1'))

In [109]:
data_Y[[1,3,2]]

tensor([[0.0670, 0.0777],
        [0.1383, 0.5785],
        [0.0628, 0.2662]], device='cuda:1')

In [79]:
data_Y[[1,2,3]]

tensor([[0.1953, 0.3120],
        [0.1027, 0.2393],
        [0.0016, 0.1836]], device='cuda:1')

In [80]:
data_Y

tensor([[0.1848, 0.1545],
        [0.1953, 0.3120],
        [0.1027, 0.2393],
        ...,
        [0.0226, 0.4141],
        [0.2555, 0.2850],
        [0.4730, 0.1915]], device='cuda:1')

In [181]:
data_X = torch.transpose(torch.tensor([[1.0,2,3,4,5]]), 0, 1).to(device)
data_Y = torch.transpose(torch.tensor([[11.0,12,13,14,15]]), 0, 1).to(device)
print(matrix_form(data_X, data_Y))
tester.u_stat_indep(data_X, data_Y)

tensor([[20.6667]], device='cuda:1')


AttributeError: 'LDPIndepTester' object has no attribute 'u_stat_indep'

In [127]:
data_X = torch.ones(10,3).to(device)
data_Y = data_X
print(matrix_form(data_X, data_Y))
tester = LDPIndepTester(device)
tester.u_stat_indep(data_X, data_Y)

tensor([[0.]], device='cuda:1')
number of calculation = 10.0
tensor(0., device='cuda:1')


In [128]:
data_X = torch.transpose(torch.tensor([[1.0, 1, 1, 1, 1,2,3,4]]), 0, 1).to(device)
data_Y = data_X
print(matrix_form(data_X, data_Y))
print(tester.u_stat_indep(data_X, data_Y))

tensor([[5.2857]], device='cuda:1')
number of calculation = 140.0
tensor(5.2857, device='cuda:1')


In [142]:
data_X = torch.transpose(torch.tensor([
    [1.0, 12, 1, 1, 1,22,3,4],[1.0, 12, 1, 1, 1,22,3,4]]
), 0, 1).to(device)
data_Y = data_X-1
print(matrix_form(data_X, data_Y))
print(tester.u_stat_indep(data_X, data_Y))

tensor([[26567.0859]], device='cuda:1')
number of calculation = 140.0
tensor(26567.1289, device='cuda:1')


In [139]:
data_X = torch.transpose(torch.tensor([[1.0, 12, 1, 1, 1,12,33,2,1,3,4,5,7,4,5,3]]), 0, 1).to(device)
data_Y = data_X-10.0
print(matrix_form(data_X, data_Y))
print(tester.u_stat_indep(data_X, data_Y))

tensor([[6293.0815]], device='cuda:1')
number of calculation = 3640.0
tensor(6293.6973, device='cuda:1')
